### **Requirements:**

In [1]:
# %pip install faker
# %pip install sqlite
# %pip install pandas
# %pip install jinja2

### **Bibliotecas:**

In [2]:
# BIBLIOTECAS
import pandas as pd
import random
import sqlite3
import datetime
from functions.criar_base import criar_base, display_df

from faker import Faker
from datetime import timedelta

### **Variáveis**

**Users**
- user_id
- name
- age
- country

**Transactions**
- user_id
- transaction_id
- transaction_date
- transaction_state
- transaction_amount

## **Criando e gerando uma base de dados aleatória através da biblioteca faker:**


In [3]:
criar_base() # Toda vez que rodada, esta função vai apagar a base de dados antigas e criar uma nova

faker_us = Faker('en-US')
faker_br = Faker('pt-BR')
faker_fr = Faker('fr-FR')

countries = ['USA', 'Brazil', 'France'] # Decidi criar uma base de dados com clientes de apenas 3 países

# Criando o dataframe que será inserido da tabela 'users'
users_df = pd.DataFrame(columns=['user_id', 'name', 'age', 'country'])

n_usuarios = 500 # Número de usuário que serão criados

# O loop vai inserir os dados diretamente no dataframe 'users_df'
for i in range(n_usuarios):
    country = random.choice(countries)
    if country == 'USA':
        faker = faker_us
    elif country == 'Brazil':
        faker = faker_br
    else:
        faker = faker_fr
        
    users_df.loc[i] = [
        i + 1,
        faker.name(),
        random.randint(18, 65),
        country
    ]

# Possíveis estados das transações
transaction_states = ['INITIATED', 'SUCCESS', 'FRAUD', 'CANCELLED']

# Criando o dataframe que será inserido da tabela 'users'
transactions_df = pd.DataFrame(columns=[
        'transaction_id',
        'user_id',
        'transaction_date',
        'transaction_state',
        'transaction_amount']
        ) 

# Gera um número de transações N (coloquei aleatório apenas para os dados variarem toda vez que o script é executado)
n_transactions = n_usuarios * random.randint(500, 1000)

# Cria uma lista auxiliar de datas de 2019 a 2024  com ano, dia, horas, minutos, segundos (formato TIMESTAMP)
random_dates = [
    (datetime.datetime(random.randint(2019, 2024), 1, 1) + timedelta(
        days=random.randint(0, 729),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59),
        seconds=random.randint(0, 59)
    )).strftime('%Y-%m-%d %H:%M:%S')

    for _ in range(n_transactions)
]

# Cria um dicionário com os dados que serão inseridos na tabela 'transactions'
transactions_data = {
    'transaction_id': range(1, n_transactions + 1),
    'user_id': [random.randint(1, 100) for _ in range(n_transactions)],
    'transaction_date': random_dates,
    'transaction_state': [random.choice(transaction_states) for _ in range(n_transactions)],
    'transaction_amount': [round(random.uniform(500, 5000), 2) for _ in range(n_transactions)]
}

# Cria um dataframe a partir do dicionário
transactions_df = pd.DataFrame(transactions_data)

# Imprime os dataframes para visualização
display(users_df)
display(transactions_df)




,user_id,name,age,country
0,1,Marine Rossi,55,France
1,2,Robert Price,24,USA
2,3,Théophile Carre,24,France
3,4,Dra. Ágatha Gomes,54,Brazil
4,5,Anaïs Louis de la Ruiz,54,France
...,...,...,...,...
495,496,Marine Navarro,37,France
496,497,João Felipe da Cunha,34,Brazil
497,498,Kimberly Hall,51,USA
498,499,Timothy Jackson,37,USA


,transaction_id,user_id,transaction_date,transaction_state,transaction_amount
0,1,41,2021-06-09 01:23:47,SUCCESS,1231.41
1,2,75,2023-01-18 16:20:01,INITIATED,2811.31
2,3,9,2019-04-22 15:40:32,INITIATED,1140.83
3,4,32,2024-03-10 03:34:44,CANCELLED,765.58
4,5,89,2021-11-25 08:43:54,CANCELLED,4008.69
...,...,...,...,...,...
298495,298496,94,2020-05-27 09:34:30,SUCCESS,4539.55
298496,298497,85,2019-02-08 10:41:05,INITIATED,1994.02
298497,298498,35,2021-11-26 08:40:53,FRAUD,4594.26
298498,298499,61,2022-08-20 17:35:53,FRAUD,2289.48


## **Criando as tabelas users e transactions**

In [4]:
# Conecta ao banco do SQLite
conn = sqlite3.connect('dbCompany')
cursor = conn.cursor()

users_data = list(users_df.itertuples(index=False, name=None))
transactions_data = list(transactions_df.itertuples(index=False, name=None))

# Inserindo users_df na tabela 'users'
cursor.executemany('INSERT INTO users (user_id, name, age, country) VALUES (?, ?, ?, ?)', users_data)

# Inserindo transactions_df na tabela 'transactions'
cursor.executemany(
                    '''
                    INSERT INTO transactions 
                    (transaction_id, user_id, transaction_date, transaction_state, transaction_amount) 
                    VALUES (?, ?, ?, ?, ?)
                    '''
                    , transactions_data
)

conn.commit()

print(f'Número de usuários inseridos: {len(users_data):,}\nNúmero de transações inseridas: {len(transactions_data):,}')

Número de usuários inseridos: 500
Número de transações inseridas: 298,500


## **Questões:**
**A.** Qual é a idade média de usuários do sistema por país

In [5]:
## QUERY A
query = '''
        SELECT country AS País, ROUND(AVG(age),2) AS "Média de idade"
        FROM users
        GROUP BY country
        ORDER BY "Média de idade" DESC
        '''

df_A = pd.read_sql(query, conn)

**B.** Qual é o país com a maior quantidade de dinheiro transacionado (considere só transações finalizadas com sucesso ou ``SUCCESS``)

In [6]:
## QUERY B
query = '''
        WITH TotalSuccess AS (
                SELECT user_id, SUM(transaction_amount) AS transacoes
                FROM transactions 
                WHERE transaction_state = 'SUCCESS'
                GROUP BY user_id
                )

        SELECT u.country AS País, 
               SUM(t.transacoes) AS "Soma das Transações (US$)"
        FROM TotalSuccess t
        LEFT JOIN users u
        ON u.user_id = t.user_id
        GROUP BY u.country
        ORDER BY "Soma das Transações (US$)" DESC
        '''

df_B = pd.read_sql(query, conn)

## Formatações no dataframe para melhor exibir as respostas no final
df_B['Soma das Transações (US$)'] = df_B['Soma das Transações (US$)'].apply(lambda x: f"{x:,.2f}")

**C.** Qual é o país com maior taxa de fraude em porcentagem respeito ao número de transações totais no país

In [7]:
## QUERY C
query = '''
        WITH
        Total AS (
            SELECT u.country, COUNT(*) as total_transacoes
            FROM transactions t
            LEFT JOIN users u
            ON t.user_id = u.user_id 
            GROUP BY u.country
        ),
        Fraudes AS (
            SELECT u.country, COUNT(*) as total_fraud
            FROM transactions t
            LEFT JOIN users u
            ON t.user_id = u.user_id
            WHERE t.transaction_state = 'FRAUD'
            GROUP BY u.country
        )
        
        SELECT 
            t.country AS País,
            ROUND(CAST(f.total_fraud AS FLOAT) / t.total_transacoes * 100, 2) as "Taxa de Fraudes (%)"
        FROM Total t
        LEFT JOIN Fraudes f
        ON t.country = f.country
        ORDER BY "Taxa de Fraudes (%)" DESC
        '''


df_C = pd.read_sql(query, conn)

**D.** Na mesma linha da pergunta anterior, responda qual é a faixa de idade de usuários
que mais cometem fraude (em percentagem).

Separe as faixas etárias em ``< 18 anos, 18-30 anos, 30 - 45 anos, 45 - 60 anos, 60 > anos``

Considerar o fato que um usuário pode ter executado várias transações, das quais poucas (ou muitas) podem ter sido fraude entre as demais.

In [8]:
## QUERY D
query = '''      
        SELECT 
            CASE 
                WHEN age < 18 THEN '< 18 anos'
                WHEN age BETWEEN 18 AND 30 THEN '18-30 anos'
                WHEN age BETWEEN 30 AND 45 THEN '30 - 45 anos'
                WHEN age BETWEEN 45 AND 60 THEN '45 - 60 anos'
                ELSE '60 > anos'
            END AS "Faixa etária",
            COUNT(*) AS "Qtd. Transações Fraudulentas",
            ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM transactions WHERE transaction_state = 'FRAUD') * 100, 2) AS "Taxa de Fraudes (%)"
        FROM users u
        JOIN transactions t ON u.user_id = t.user_id
        
        WHERE t.transaction_state = 'FRAUD'
        GROUP BY "Faixa etária"
        ORDER BY "Faixa etária" ASC
        '''

df_D = pd.read_sql(query, conn)

## Formatações no dataframe para melhor exibir as respostas no final
df_D["Qtd. Transações Fraudulentas"] = df_D["Qtd. Transações Fraudulentas"].apply(lambda x: f"{x:,.0f}")

**E.** Imagine que a camada executiva da empresa dona do sistema, precisa criar um Dashboard para monitorar o estado das transações nos últimos 3 dias.

Criar uma query SQL que calcule:

1. ``Número e dinheiro das transações não finalizadas``

2. ``Número e dinheiro de transações finalizadas com sucesso (SUCCESS)``

3. ``Número e dinheiro de transações canceladas (CANCELLED)``

4. ``Número e dinheiro de fraudes (FRAUD)``

Agrupado por país e nos 3 dias anteriores de quando o executivo da empresa consulte seu Dashboard.

In [9]:
## QUERY E:
query = '''      
        WITH TabelaAux AS (
            SELECT 
                u.country,
                t.transaction_state,
                COUNT(*) AS transaction_count,
                SUM(t.transaction_amount) AS transaction_amount
            FROM transactions t
            JOIN users u ON u.user_id = t.user_id

            WHERE t.transaction_date <= DATE('now', '-3 day')
                -- SQLite não possui as funções CURDATE() e INTERVAL
                -- WHERE t.transaction_date <= CURDATE() - INTERVAL 3 DAY)

            GROUP BY u.country, t.transaction_state
        )
        SELECT 
            country,
            SUM(CASE WHEN transaction_state = 'INITIATED' THEN transaction_count ELSE 0 END) AS "N. Não Finalizadas",
            SUM(CASE WHEN transaction_state = 'INITIATED' THEN transaction_amount ELSE 0 END) AS "Soma Não Finalizadas",
            
            SUM(CASE WHEN transaction_state = 'SUCCESS' THEN transaction_count ELSE 0 END) AS "N. Finalizadas",
            SUM(CASE WHEN transaction_state = 'SUCCESS' THEN transaction_amount ELSE 0 END) AS "Soma Finalizadas",
            
            SUM(CASE WHEN transaction_state = 'CANCELLED' THEN transaction_count ELSE 0 END) AS "N. Canceladas",
            SUM(CASE WHEN transaction_state = 'CANCELLED' THEN transaction_amount ELSE 0 END) AS "Soma Canceladas",
            
            SUM(CASE WHEN transaction_state = 'FRAUD' THEN transaction_count ELSE 0 END) AS "N. Fraudulentas",
            SUM(CASE WHEN transaction_state = 'FRAUD' THEN transaction_amount ELSE 0 END) AS "Soma Fraudulentas"
        FROM TabelaAux
        GROUP BY country
        '''

df_E = pd.read_sql(query, conn)

## Formatações no dataframe para melhor exibir as respostas no final
df_E["N. Não Finalizadas"] = df_E["N. Não Finalizadas"].apply(lambda x: f"{x:,.0f}")
df_E["N. Finalizadas"] = df_E["N. Finalizadas"].apply(lambda x: f"{x:,.0f}")
df_E["N. Canceladas"] = df_E["N. Canceladas"].apply(lambda x: f"{x:,.0f}")
df_E["N. Fraudulentas"] = df_E["N. Fraudulentas"].apply(lambda x: f"{x:,.0f}")

df_E["Soma Não Finalizadas"] = df_E["Soma Não Finalizadas"].apply(lambda x: f"{x:,.2f}")
df_E["Soma Finalizadas"] = df_E["Soma Finalizadas"].apply(lambda x: f"{x:,.2f}")
df_E["Soma Canceladas"] = df_E["Soma Canceladas"].apply(lambda x: f"{x:,.2f}")
df_E["Soma Fraudulentas"] = df_E["Soma Fraudulentas"].apply(lambda x: f"{x:,.2f}")


### **Respostas:**

In [10]:
# IMPRIMIR OS RESULTADOS

print('Questão A:')
display(df_A)
print('Questão B:')
display(df_B)
print('Questão C:')
display(df_C)
print('Questão D:')
display(df_D)
print('Questão E:')
display(df_E)


# conn.close()

Questão A:


,País,Média de idade
0,USA,42.37
1,France,41.39
2,Brazil,40.94


Questão B:


,País,Soma das Transações (US$)
0,France,"81,303,109.57"
1,Brazil,"65,133,652.17"
2,USA,"57,353,139.63"


Questão C:


,País,Taxa de Fraudes (%)
0,Brazil,25.16
1,France,24.99
2,USA,24.91


Questão D:


,Faixa etária,Qtd. Transações Fraudulentas,Taxa de Fraudes (%)
0,18-30 anos,"23,858",31.94
1,30 - 45 anos,"20,100",26.91
2,45 - 60 anos,"24,641",32.99
3,60 > anos,"6,086",8.15


Questão E:


,country,N. Não Finalizadas,Soma Não Finalizadas,N. Finalizadas,Soma Finalizadas,N. Canceladas,Soma Canceladas,N. Fraudulentas,Soma Fraudulentas
0,Brazil,"22,212","60,979,561.40","22,151","60,844,913.71","22,478","61,852,340.55","22,400","61,544,913.55"
1,France,"27,929","77,243,479.07","27,750","76,069,187.95","28,058","77,290,509.65","27,932","76,828,621.99"
2,USA,"19,643","54,115,644.41","19,532","53,505,146.98","19,839","54,964,518.80","19,595","54,074,936.70"
